In [211]:
import holidays
import re
import calendar
from calendar import monthrange
from datetime import timedelta
from collections import Counter
from datetime import datetime

def get_hours(ISO, peak_type, period):
    def dst_adjust(company:str, first_dst:bool, second_dst:bool, hours:int): #pass a function to adjust dst-affected hours
        if company != "MISO": #MISO doesn't have daylight savings
            if first_dst: #first dst accounts for the dst in March, minus 1 hour 
                return hours-1
            elif second_dst: #second dst, plus 1 hour back
                return hours+1
        return hours
    eastern = ['PJMISO', 'MISO', 'ERCOT', 'SPPISO', 'NYISO']
    western = ['WECC', 'CAISO']
    if ISO not in eastern and ISO not in western:
        raise Exception('invalid iso')
    peak_lst = ['onpeak', 'offpeak', 'flat', '2x16H', '7x8']
    if peak_type not in peak_lst:
        raise Exception('invalid peak type')
    start = end = ''#start date and end date for the period initialized
    quarters = [('1-1', '3-31'), ('4-1', '6-30'), ('7-1', '9-30'), ('10-1', '12-31')] #start and end dates of each quarter
    month_start = {'Jan':'1-1', 'Feb':'2-1','Mar':'3-1','Apr':'4-1','May':'5-1','June':'6-1', 'July':'7-1', \
                  'Aug':'8-1', 'Sep':'9-1', 'Oct':'10-1', 'Nov':'11-1', 'Dec':'12-1'} #start date of each month
    month_num =  {'Jan': 1,'Feb': 2,'Mar': 3,'Apr': 4,'May': 5,'June': 6,'July': 7,'Aug': 8,'Sep': 9,'Oct': 10,\
                      'Nov': 11,'Dec': 12} #corresponding month number
    NERC_holidays = {"New Year's Day":'', 'Memorial Day':'', 'Independence Day':'', 'Labor Day':'', \
                     'Thanksgiving':'', 'Christmas Day':''}
    NERC_holidays_count = {'weekday':0, 'Sat':0} #Sundays and NERC holidays have overlapping hours, no need to count Sundays
    try: #try splitting the period string to account for 4 types of valid input
        split_period = re.split('(\d+)', period)
        year = split_period[1]
        if period[-1] == 'A': #annual case
            start = datetime.strptime(year+'-1-1','%Y-%m-%d').date()
            end = datetime.strptime(year+'-12-31','%Y-%m-%d').date()
        elif period[-2] == 'Q': #quarter case
            start = datetime.strptime(year+'-'+quarters[int(period[-1])-1][0],'%Y-%m-%d').date()
            end = datetime.strptime(year+'-'+quarters[int(period[-1])-1][1],'%Y-%m-%d').date()
        elif split_period[-1] in month_num: #month case
            start = datetime.strptime(year+'-'+month_start[split_period[-1]],'%Y-%m-%d').date()
            end = start+timedelta(monthrange(int(year), month_num[split_period[-1]])[1]-1)
        elif split_period[2] == split_period[4] == '-': #one-day case
            start = datetime.strptime(period,'%Y-%m-%d').date()
            end = start
    except:
        raise Exception('Invalid period entered')
    for date, holiday in holidays.UnitedStates(years = int(year)).items():
        if (holiday in NERC_holidays) and (start <= date <= end) and (date.strftime('%a') != 'Sun'):
            if date.strftime('%a') != 'Sat':
                NERC_holidays_count['weekday'] += 1 
            else:
                NERC_holidays_count['Sat'] += 1
    dst1 = calendar.Calendar(6).monthdatescalendar(int(year), 3)[2][0] #this assigns the date of the second Sunday in March
    dst2 = calendar.Calendar(6).monthdatescalendar(int(year), 11)[1][0] #the first Sunday in November
    dst1_on = start <= dst1 <= end and not(start <= dst2 <= end)# period includes the first dst only
    dst2_on = start <= dst2 <= end and not(start <= dst1 <= end)# period includes the second dst only
    days_count = Counter() #count the days in a week over the year
    for i in range((end - start).days+1):
        days_count[(start + timedelta(i)).strftime('%a')] += 1
    hours = 0
    if peak_type == 'flat':
        hours += 24*sum(days_count[day] for day in days_count)
        hours = dst_adjust(ISO, dst1_on, dst2_on, hours)
    elif peak_type == 'onpeak':
        hours += 16*(days_count['Mon']+days_count['Tue']+days_count['Wed']+days_count['Thu']+days_count['Fri'])
        hours -= 16*NERC_holidays_count['weekday'] #minus the holidays
        if ISO in western:
            hours += 16*days_count['Sat'] #Saturday is a weekday for Western
            hours -= 16*NERC_holidays_count['Sat']
    elif peak_type == 'offpeak':
        hours += 8*(days_count['Mon']+days_count['Tue']+days_count['Wed']+days_count['Thu']+days_count['Fri'])\
        +24*days_count['Sun']
        hours += 16*NERC_holidays_count['weekday']
        if ISO in eastern:
            hours += 24*days_count['Sat']
        else:
            hours += 16*NERC_holidays_count['Sat']
            hours += 8*days_count['Sat']
        hours = dst_adjust(ISO, dst1_on, dst2_on, hours)
    elif peak_type == '2x16H':
        hours += 16*(days_count['Sun']+NERC_holidays_count['weekday'])
        if ISO in eastern:
            hours += 16*days_count['Sat'] #Weekends include Saturday for Eastern
        else:
            hours += 16*NERC_holidays_count['Sat']
    elif peak_type == '7x8':
        hours += 8*sum(days_count[day] for day in days_count)
        hours = dst_adjust(ISO, dst1_on, dst2_on, hours)
    return [ISO, peak_type, start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), hours]

In [224]:
get_hours('ERCOT', 'onpeak', '2019May')

['ERCOT', 'onpeak', '2019-05-01', '2019-05-31', 352]